# Data Quality Assessment für die Bachelorarbeit: Schadenserkennung bei Büchern mit machine learning

In [226]:
import pandas as pd
import matplotlib

In [227]:
# einlesen der Daten von json
data = pd.read_json("2024-02-21-Schadenserkennung-Annotationen.json")
data = data.set_index("id")

## Verwerfen von nicht brauchbaren Daten

In [228]:
# verwerfen von nicht zu verwendenen Daten
#
#   created_at und updated_at geben keine informationen über den inhalt des bildes bzw der annotation
#   lead_time interessiert nicht
#   annotator_id ist die id + 54
#   annotator ist immer 1
#   choice hat 974/1081 nullwerte (Nan)
#   
# Die restlichen Werte können relevant sein für das trainieren des algorithmus

data_dropped = data.drop(["created_at", "updated_at", "lead_time", "annotation_id", "annotator", "choice"], axis=1)

## Kategorie "label" weiter aufteilen
Extrahieren der einzelnen Schadensarten und festlegen ob es überhaupt schäden hat.

In [229]:
label = pd.json_normalize(data_dropped["label"])
total = ['Schimmel', 'Wasser', 'Sonstiges', 'Schäden', 'Schmutz', 'Schädlinge']

# Add column for every category used
for x in total:
    data_dropped[x] = False
data_dropped["hat_schäden"] = None

# True where category is true
# rows
for x in range(label.shape[0]):
    rowx = set()
    # colums
    for y in range(label.shape[1]):
        if label[y][x] != None:
            rowx.add(label[y][x]["polygonlabels"][0])
    if label[0][x] == None:
        data_dropped.loc[x+1, "hat_schäden"]= False
    else:
        for s in rowx:
            data_dropped.loc[x+1, s] = True
        data_dropped.loc[x+1, "hat_schäden"] = True

## Datentyps kontrollieren
image ist ein objekt, da strings in pandas so gespeichert werden.

label ist ein objekt da es gemischte Datentypen enthält: strings, int, float, bool

In [230]:
# change "hat_schäden" da es nur True und Fale drin hat
data_preprocessed = data_dropped.astype({"hat_schäden" : "bool"})

## Auf None/Nan kontrollieren
Wenn diese in "image" auftreten --> löschen

In [231]:
# False = keine None/Nan
nonetest = data_preprocessed["image"].isnull().any()

if nonetest == True:
    # drop rows with None only
    data_preprocessed.dropna(axis=0, how="all", inplace=True)
    # drop rows with none in image, without areference it is impossible to get to the right picture
    data_preprocessed.dropna(axis=0, subset=["image"], inplace=True)

## Mengen und verschiedene Anhaltspunkte

In [232]:
data_preprocessed["hat_schäden"].value_counts()

hat_schäden
True     882
False    199
Name: count, dtype: int64

Es hat 882 Einträge die schäden aufweisen und 199 die keine haben

In [251]:
truthlist = {}
for x in total:
    truthlist[x] = data_preprocessed[x].value_counts()[True]
print(truthlist)

{'Schimmel': 10, 'Wasser': 274, 'Sonstiges': 402, 'Schäden': 237, 'Schmutz': 871, 'Schädlinge': 4}


In [256]:
data_preprocessed[total].value_counts()

Schimmel  Wasser  Sonstiges  Schäden  Schmutz  Schädlinge
False     False   False      False    True     False         234
                  True       False    True     False         207
                  False      False    False    False         199
          True    False      False    True     False         100
                  True       False    True     False          87
          False   False      True     True     False          80
                  True       True     True     False          69
          True    False      True     True     False          52
                  True       True     True     False          28
          False   True       False    False    False           6
True      False   False      False    True     False           3
                  True       False    True     False           2
                  False      True     True     False           2
False     True    False      False    True     True            2
          False   False      Tru

# TODO: 
1. verteilung der einzelnen schadenstypen, menge, min, max usw
3. verteilung von: hat schäden und hat keine


Zeile 216 hat die meisten schadenseinträge, gesamt 37